In [15]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("../input"))

pd.set_option('display.max_columns', None)

['train_transaction.csv', 'train_identity.csv', 'sample_submission.csv', 'standalone-train-and-test-preprocessing', 'test_identity.csv', 'test_transaction.csv']


In [2]:
from sklearn import preprocessing
import xgboost as xgb

In [3]:
train_transaction = pd.read_csv('../input/train_transaction.csv', index_col='TransactionID')
test_transaction = pd.read_csv('../input/test_transaction.csv', index_col='TransactionID')

train_identity = pd.read_csv('../input/train_identity.csv', index_col='TransactionID')
test_identity = pd.read_csv('../input/test_identity.csv', index_col='TransactionID')

sample_submission = pd.read_csv('../input/sample_submission.csv', index_col='TransactionID')

In [4]:
train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
test = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)

print(train.shape)
print(test.shape)

y_train = train['isFraud'].copy()

# Drop target, fill in NaNs
X_train = train.drop('isFraud', axis=1)
X_test = test.copy()
X_train = X_train.fillna(-999)
X_test = X_test.fillna(-999)

(590540, 433)
(506691, 432)


In [5]:
del train, test, train_transaction, train_identity, test_transaction, test_identity

In [6]:
# cater_list = []
categorical_list = ['ProductCD', 'addr1', 'addr2', 'P_emaildomain', 'R_emaildomain',
            'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1','addr2',
             'M1','M2','M3','M4','M5','M6','M7','M8','M9',
              'id_12','id_13','id_14','id_15','id_16','id_17','id_18','id_19','id_20',
              'id_21','id_22','id_23','id_24','id_25','id_26','id_27','id_28','id_29','id_30',
              'id_31','id_32','id_33','id_34','id_35','id_36','id_37','id_38',
              'DeviceType','DeviceInfo'
             ]

In [7]:
# Label Encoding
for f in X_train.columns:
    if X_train[f].dtype=='object' or X_test[f].dtype=='object' or f in categorical_list: 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f].values) + list(X_test[f].values))
        X_train[f] = lbl.transform(list(X_train[f].values))
        X_test[f] = lbl.transform(list(X_test[f].values))   

In [8]:
print(X_train.shape)
print(X_test.shape)

(590540, 432)
(506691, 432)


In [18]:
X_train['id_23'].unique()

array([0, 3, 1, 2])

In [10]:
# clf = xgb.XGBClassifier(n_estimators=500,
#                         n_jobs=-1,
#                         max_depth=9,
#                         learning_rate=0.05,
#                         subsample=0.9,
#                         colsample_bytree=0.9,
#                         missing=-999,
#                        )

In [11]:
## 10 mins
# %time clf.fit(X_train, y_train) #29.7G RAM

In [12]:
clf = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=9,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    missing=-999,
    random_state=2019,
    tree_method='gpu_hist'  # THE MAGICAL PARAMETER
)

In [13]:
%time clf.fit(X_train, y_train)

CPU times: user 26.9 s, sys: 6.07 s, total: 32.9 s
Wall time: 29 s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=-999, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=2019,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.9, tree_method='gpu_hist', verbosity=1)

In [14]:
sample_submission['isFraud'] = clf.predict_proba(X_test)[:,1]
sample_submission.head()
# sample_submission.to_csv('../submissions/simple_xgboost.csv')

,isFraud
TransactionID,
3663549,0.005154
3663550,0.002690
3663551,0.003020
3663552,0.003573
3663553,0.001408
